In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np

openai.api_key = API_key
# modellist = openai.Model.list()
# print('All models:')
# for i in range(len(openai.Model.list()['data'])):
#     print(modellist['data'][i]['id'])
from GptPrompter import *
from AutoReasoner import *
# ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

In [7]:
demo_num = 0
at_index = None

def parallel_codex_func(i):
    try:
        codex_prompter = CodexSQL(training_df.iloc[i]['id'], 
                                  training_df.iloc[i]['utterance'], 
                                  training_df.iloc[i]['context'], 
                                  training_df.iloc[i]['targetValue'], 
                                  base_path='./dataset/WikiTableQuestions/',
                                  demo_file='./dataset/WikiTableQuestions/examples-sql.txt'
                                 )
        codex_prompter._gen_codex_prompt(demo_num=demo_num, at_index=at_index)
        codex_prompter._get_gpt_prediction()
        if codex_prompter.gpt_error is None:
            codex_prompter._evaluate_result(verbose=False)
        return codex_prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }

def parallel_gpt_answer_func(i):
    try:
        prompter = GptAnswer(training_df.iloc[i]['id'], 
                             training_df.iloc[i]['utterance'], 
                             training_df.iloc[i]['context'], 
                             training_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/',
                             demo_file='./dataset/WikiTableQuestions/examples.txt')
        prompter._gen_gpt_prompt(demo_num=demo_num, at_index=at_index)
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }

def parallel_codex_answer_func(i):
    try:
        prompter = GptAnswer(training_df.iloc[i]['id'], 
                             training_df.iloc[i]['utterance'], 
                             training_df.iloc[i]['context'], 
                             training_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/',
                             demo_file='./dataset/WikiTableQuestions/examples.txt')
        prompter.model = "davinci-codex-002-msft"
        prompter._gen_gpt_prompt(demo_num=demo_num, at_index=at_index)
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    
def parallel_gpt_instruct_func(i):
    try:
        prompter = GptInstruct(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
        )
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }

def parallel_GptAnswerReason_func(i):
    try:
        prompter = GptAnswerReason(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
            demo_file='./dataset/WikiTableQuestions/reasoning-examples.txt'
        )
        prompter._gen_gpt_prompt(demo_num=demo_num)
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }   

def parallel_CodexAnswerReason_func(i):
    try:
        prompter = CodexAnswerReason(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
            demo_file='./dataset/WikiTableQuestions/reasoning-examples.txt'
        )
        prompter.model = "davinci-codex-002-msft"
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }   
    
def parallel_CodexSQLReason_func(i):
    try:
        prompter = CodexSQLReason(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
            demo_file='./dataset/WikiTableQuestions/reasoning-examples-sql.txt'
        )
        prompter._gen_gpt_prompt(demo_num=demo_num)
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    
maxLimit = np.inf 
# maxLimit = 20
splits = ['pristine-unseen-tables']

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 40
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_codex_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     # logs = Parallel(n_jobs=n_threads)(delayed(parallel_codex_func)(i) for i in tqdm(range(5)))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexSQL_fewShot_demoNum{demo_num}_atIndex{at_index}_results_{split}.json', 'w'))

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 40
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_gpt_answer_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/results/GptAnswer_fewShot_demoNum{demo_num}_atIndex{at_index}_results_{split}.json', 'w'))

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 40
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_codex_answer_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     # logs = Parallel(n_jobs=n_threads)(delayed(parallel_gpt_answer_func)(i) for i in tqdm(range(5)))
    # json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexAnswer_fewShot_demoNum{demo_num}_atIndex{at_index}_results_{split}.json', 'w'))
    
# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 20
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_gpt_instruct_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/GptInstruct_results_{split}.json', 'w'))

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 40
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_GptAnswerReason_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/results/GptAnswerReason_results_{split}.json', 'w'))

for split in splits:
    training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
    print(training_df.shape)
    print(training_df.iloc[0])
    n_threads = 20
    # maxLimit = 1
    from joblib import Parallel, delayed
    logs = Parallel(n_jobs=n_threads)(delayed(parallel_CodexAnswerReason_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
    json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexAnswerReason_results_{split}.json', 'w'), indent=4)

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     n_threads = 20
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads)(delayed(parallel_CodexSQLReason_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexSQLReason_results_{split}.json', 'w'))

# ft=None
# training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='warn')
# parallel_CodexAnswerOrderExplorer_func(12)

(4344, 4)
id                                                          nu-0
utterance      which country had the most cyclists finish wit...
context                                      csv/203-csv/733.csv
targetValue                                                Italy
Name: 0, dtype: object


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [06:38<00:00, 10.90it/s]


nu-138 The server is overloaded or not ready yet.
nu-514 This model's maximum context length is 8193 tokens, however you requested 9774 tokens (9646 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
nu-573 This model's maximum context length is 8193 tokens, however you requested 13195 tokens (13067 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
nu-1365 The server is overloaded or not ready yet.
nu-3092 This model's maximum context length is 8193 tokens, however you requested 9018 tokens (8890 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
nu-3336 The server is overloaded or not ready yet.
nu-147 The server is overloaded or not ready yet.
nu-2047 This model's maximum context length is 8193 tokens, however you requested 11525 tokens (11397 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
nu-2243 This model's maximum context

In [6]:
with open(f'/mnt/text2sql/dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_pristine-unseen-tables.json', 'r') as f:
    logs = json.load(f)

with open(f'/mnt/text2sql/dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_pristine-unseen-tables_to_combine.json', 'r') as f:
    logs_to_combine = json.load(f)

assert len(logs) == len(logs_to_combine)
for i in range(len(logs)):
    assert logs[i]['id'] == logs_to_combine[i]['id']
    if 'all_df_permutation_results' in logs[i] and 'all_df_permutation_results' in logs_to_combine[i]:
        logs[i]['all_df_permutation_results'].update(logs_to_combine[i]['all_df_permutation_results'])

with open(f'/mnt/text2sql/dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_pristine-unseen-tables_combined.json', 'w') as f:
    json.dump(logs, f)
    

# CodexAnswerOrderExplorer

In [3]:
ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')
training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
print("Loading embeddings")
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

Loading embeddings


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14149/14149 [00:20<00:00, 703.18it/s]


In [5]:
def parallel_CodexAnswerOrderExplorer_func(i):
    try:
        prompter = CodexAnswerOrderExplorer(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/'
        )
        prompter._gen_all_table_permutations(ft=ft)
        prompter._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._explore_all_dataframe_permutations()
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    
def parallel_CodexAnswerRandShuffle_func(i):
    try:
        prompter = CodexAnswerRandShuffle(
            training_df.iloc[i]['id'], 
            training_df.iloc[i]['utterance'], 
            training_df.iloc[i]['context'], 
            training_df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/'
        )
        prompter._gen_all_table_permutations(ft=ft)
        prompter._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._explore_all_dataframe_permutations()
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': training_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    
maxLimit = np.inf 
# maxLimit = 20
splits = ['pristine-unseen-tables']
n_threads = 20

# for split in splits:
#     training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
#     print(training_df.shape)
#     print(training_df.iloc[0])
#     from joblib import Parallel, delayed
#     logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_CodexAnswerOrderExplorer_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
#     json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_{split}_complete.json', 'w'))

for split in splits:
    training_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/{split}.tsv', sep='\t', on_bad_lines='warn')
    print(training_df.shape)
    print(training_df.iloc[0])
    from joblib import Parallel, delayed
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_CodexAnswerRandShuffle_func)(i) for i in tqdm(range(min(maxLimit, training_df.shape[0]))))
    json.dump(logs, open(f'./dataset/WikiTableQuestions/results/CodexAnswerRandShuffle_func_results_{split}_complete.json', 'w'))

(4344, 4)
id                                                          nu-0
utterance      which country had the most cyclists finish wit...
context                                      csv/203-csv/733.csv
targetValue                                                Italy
Name: 0, dtype: object


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [28:59<00:00,  2.50it/s]


# cos few shot

In [7]:
import time

# model = 'text-davinci-002'
# model = 'text-davinci-003'
model = 'davinci-codex-002-msft'

ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

def parallel_gpt_answer_dynamic_few_shot_func(i):
    try:
        prompter = GptAnswer(test_df.iloc[i]['id'], 
                             test_df.iloc[i]['utterance'], 
                             test_df.iloc[i]['context'], 
                             test_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/')
        prompter.model = model
        prompter._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        log = prompter._log_dict()
    except Exception as e:
        log = {
            'id': test_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    if model == 'text-davinci-003':
        time.sleep(5)
    return log

training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
print("Loading embeddings")
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
maxLimit = float('inf')
# maxLimit = 30
n_threads = 1
from joblib import Parallel, delayed
res = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_gpt_answer_dynamic_few_shot_func)(i) for i in tqdm(range(min(maxLimit, test_df.shape[0]))))
json.dump(res, open(f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_results_pristine-unseen-tables.json', 'w'))

Loading embeddings


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [45:09<00:00,  1.60it/s]


In [9]:
# test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
# parallel_gpt_answer_dynamic_few_shot_func(1000)

In [3]:
execution_match_cnt = 0
execution_err_cnt = 0
gpt_error = 0

for i, q in enumerate(logs):
    if 'execution_match' in q and q['execution_match']==True:
        execution_match_cnt += 1
    if 'execution_err' in q and q['execution_err'] is not None:
        execution_err_cnt += 1
    if 'gpt_error' in q and q['gpt_error'] is not None:
        gpt_error += 1
        # print(i)
        if gpt_error == 1:
            print(q['gpt_error'])
    # if 'target_value' in q and q['predicted_value'] is not None and q['target_value'] != q['predicted_value'] and q['target_value'] in q['predicted_value']:
    #     print(f"Prediction: {q['predicted_value']}, target: {q['target_value']}")
print(f"Execution match: {execution_match_cnt/len(logs):.2f}")
print(f"Execution err: {execution_err_cnt/len(logs):.2f}")
if gpt_error > 0:
    print(f"GPT error: {gpt_error/len(logs):.2f}")  

This model's maximum context length is 4097 tokens, however you requested 7490 tokens (7362 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
Execution match: 0.46
Execution err: 0.00
GPT error: 0.02


# Write prediction file

## single prediction

In [6]:
with open(f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_results_pristine-unseen-tables.json', 'r') as f:
    logs = json.load(f)
predictions = ''
for q in logs:
    qid = q['id'].replace(' ', '')
    if 'predicted_value' in q:
        result = str(q['predicted_value']).replace('\n', ' ').replace('\t', ' ')
    else:
        result = ''
    predictions += f"{qid}\t{result}\n"
with open(f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_results_pristine-unseen-tables_predictions.tsv', 'w') as g:
    g.write(predictions)

# import subprocess
# out = subprocess.check_output(['python2', './dataset/WikiTableQuestions/validate-and-evaluate.py', 
#                                '-d', './dataset/WikiTableQuestions/', 
#                                '-o', './dataset/WikiTableQuestions/results/output.json',
#                                'test', 
#                                f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_results_pristine-unseen-tables_predictions.tsv'])
# print(out)

b'Dataset version: "Version 1.0.2 (October 4, 2016)"\nusing test split: pristine-unseen-tables.tsv\nReading split file:  pristine-unseen-tables.tsv\nRead 4344 example IDs\nExample ids:  [\'nu-0\', \'nu-1\', \'nu-2\', \'nu-3\', \'nu-4\', \'nu-5\', \'nu-6\', \'nu-7\', \'nu-8\', \'nu-9\']\nUsing prediction file:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_results_pristine-unseen-tables_predictions.tsv\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged\nReading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged\nRead 22033 examples\nReading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_results_pristine-unseen-tables_predictions.tsv\nExamples: 4344\nCorrect: 893\nAccuracy: 0.2056\n\n'


## multiple predictions

In [9]:
with open(f'/mnt/text2sql/dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_pristine-unseen-tables.json', 'r') as f:
    logs = json.load(f)
predictions = ''
for q in logs:
    qid = q['id'].replace(' ', '')
    predictions += f'{qid}'
    if 'all_df_permutation_results' in q:
        for reason in q['all_df_permutation_results']:
            result = q['all_df_permutation_results'][reason]
            predictions += f"\t{result}"
    predictions += '\n'
with open(f'./dataset/WikiTableQuestions/results/CodexAnswerOrderExplorer_results_pristine-unseen-tables_predictions.tsv', 'w') as g:
    g.write(predictions)

In [8]:
q

{'id': 'nu-228',
 'uncaught_err': 'Error tokenizing data. C error: EOF inside string starting at row 11'}

# Result quality summary

In [4]:
['CodexSQL', 'CodexSQL_fewShot', 'GptAnswer', 'GptAnswer_fewShot', 'GptAnswerReason', 'CodexSQLReason']
for method in ['CodexSQL_fewShot', 'GptAnswer_fewShot']:
    split = 'pristine-unseen-tables'
    with open(f'./dataset/WikiTableQuestions/results/{method}_demoNum{demo_num}_atIndex{at_index}_results_{split}.json', 'r') as f:
        logs = json.load(f)
    execution_match_cnt = 0
    execution_err_cnt = 0
    gpt_error = 0
    for q in logs:
        if 'execution_match' in q and q['execution_match']==True:
            execution_match_cnt += 1
        if 'execution_err' in q and q['execution_err'] is not None:
            execution_err_cnt += 1
        if 'gpt_error' in q and q['gpt_error'] is not None:
            gpt_error += 1
            if gpt_error == 1:
                print(q['gpt_error'])
    print(f"{method}: Execution match: {execution_match_cnt/len(logs):.2f}")
    print(f"Execution err: {execution_err_cnt/len(logs):.2f}")
    if gpt_error > 0:
        print(f"GPT error: {gpt_error/len(logs):.2f}")    

This model's maximum context length is 8193 tokens, however you requested 10149 tokens (8149 in your prompt; 2000 for the completion). Please reduce your prompt; or completion length.
CodexSQL_fewShot: Execution match: 0.49
Execution err: 0.08
Execution err: 0.02
This model's maximum context length is 4097 tokens, however you requested 8133 tokens (8005 in your prompt; 128 for the completion). Please reduce your prompt; or completion length.
GptAnswer_fewShot: Execution match: 0.46
Execution err: 0.00
Execution err: 0.04


# format prediction 

In [10]:
configs = ['CodexSQL', 'CodexSQL_fewShot', 'GptAnswer', 'GptAnswer_fewShot', 'GptAnswerReason', 'CodexSQLReason']
configs = ['CodexSQL_fewShot', 'GptAnswer_fewShot']

import subprocess
for method in configs:
    for split in splits:
        with open(f'./dataset/WikiTableQuestions/results/{method}_demoNum{demo_num}_atIndex{at_index}_results_{split}.json', 'r') as f:
            logs = json.load(f)
        predictions = ''
        for q in logs:
            qid = q['id'].replace(' ', '')
            if 'predicted_value' in q:
                result = str(q['predicted_value']).replace('\n', ' ').replace('\t', ' ')
            else:
                result = ''
            predictions += f"{qid}\t{result}\n"
        with open(f'./dataset/WikiTableQuestions/results/{method}_demoNum{demo_num}_atIndex{at_index}_{split}_predictions.tsv', 'w') as g:
            g.write(predictions)
    
    # cmd = ' '.join(['python2', './dataset/WikiTableQuestions/validate-and-evaluate.py', '-d', './dataset/WikiTableQuestions/', 'test', 
    #                                f'./dataset/WikiTableQuestions/results/{method}_demoNum{demo_num}_atIndex{at_index}_results_{split}.json'])
    # cmd = ' '.join(['python2', 'validate-and-evaluate.py', '-d', './', 'test', 
    #                                f'./results/{method}_demoNum{demo_num}_atIndex{at_index}_{split}_predictions.tsv'])
    # print(cmd)
    out = subprocess.check_output(['python2', './dataset/WikiTableQuestions/validate-and-evaluate.py', '-d', './dataset/WikiTableQuestions/', 'test', 
                                   f'./dataset/WikiTableQuestions/results/{method}_demoNum{demo_num}_atIndex{at_index}_{split}_predictions.tsv'])
    print(method)
    print(out)
    # python2 validate-and-evaluate.py -d ./ test ./results/CodexSQL_fewShot_pristine-unseen-tables_predictions.tsv
    

JSONDecodeError: Expecting value: line 1 column 264 (char 263)

# Combine ensumble

In [13]:
def selectMajority(l):
    from collections import Counter
    c = Counter(l)
    value, _ = c.most_common()[0]
    return value

configs = ['GptAnswer_fewShot_demoNumNone_atIndex{}', 'CodexSQL_fewShot_demoNumNone_atIndex{}']
for method in configs:
    answer_dicts = []
    for index in [0, 1, 2,]:
        answer_dicts.append(json.load(open(f'./dataset/WikiTableQuestions/results/{method.format(index)}_results_pristine-unseen-tables.json', 'r')))
    predictions = ''
    for i, q in enumerate(answer_dicts[0]):
        qid = q['id'].replace(' ', '')
        current_predictions = []
        for d in answer_dicts:
            if 'predicted_value' in d[i]:
                current_predictions.append(str(d[i]['predicted_value']).replace('\n', ' ').replace('\t', ' '))
            else:
                current_predictions = ['']
        # print(selectMajority(current_predictions))
        predictions += f"{qid}\t{selectMajority(current_predictions)}\n"
    with open(f'./dataset/WikiTableQuestions/results/{method}_ensumble_pristine-unseen-tables_predictions.tsv', 'w') as g:
        g.write(predictions)